In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import time
import pyarrow as pa
import pyarrow.parquet as pq
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
!pip install feather-format >> none
!pip install faiss-cpu --no-cache
!pip install implicit
import implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 18.1 MB/s eta 0:00:0000:0100:01


From csv-file from previous script I choose sites that contains symbol � (unrecognized cyrillic letters)

In [4]:
site_data = pd.read_csv('data/sites.csv', header=None)
site_data.columns = ['position', 'url', 'title', 'description', 'keywords']
site_data = site_data[site_data['title'].str.contains('[�]+', na=False)]
site_data

0                                ad.adriver.ru
1                                    apple.com
2                       avatars.mds.yandex.net
3                  googleads.g.doubleclick.net
5                                  i.ytimg.com
                           ...                
322861818                        poliglot1.com
322872650                     reservemaster.ru
322882531                       uploadhive.com
322883336            kuzma2012.livejournal.com
322888794    stroitelnye-materialy-v-kurske.ru
Name: url_host, Length: 199683, dtype: category
Categories (199683, object): ['-1', '0--stranger-livejournal-com.turbopages.org', '0-1.ru', '0-34.ru', ..., 'юбилей-на-бис.рф', 'южныйокруг.рф', 'явернусь.рф', 'яркнига24.рф']

In [ ]:
no_lang = site_data['url']
no_lang

0               razdeti.ru
1             vikidalka.ru
2             m.iamcook.ru
3            pozdravok.com
4                 12apr.su
               ...        
14504        100megabit.ru
14505    tlt.100megabit.ru
14506       m.danji100.com
14507         100letvvs.ru
14508             555hf.tv
Name: url, Length: 14509, dtype: object

Function for writing sites data to csv-file

In [ ]:
import csv
def csv_writer(data):
    with open('data/sites-cp1251.csv', 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(data)

Function like in previous script but with cp1251 encoding

In [ ]:
%%time
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
from urllib3.exceptions import LocationParseError

def parse_url(url):
    url_data = np.empty([5], dtype=object)
    url_data[0] = url[0]
    url_data[1] = url[1]
    try:
        if (url[1] == 'sanstv.ru'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 't.lordfilms-filmy.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'z.lordfilms-film.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'n.lordfilm-smotret.one'):
            csv_writer(url_data)       
            del(url_data)
            return
        if (url[1] == 'z.lordfilmy-film.online'):
            csv_writer(url_data)       
            del(url_data)
            return
        html = requests.get(f"http://{url[1]}", timeout=6, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 
                                                           'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
                                                           'Accept-Encoding': 'gzip, deflate, br',
                                                           'Accept-Language':  'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
                                                           'Cookie': 'vid=1b448c0013086e08644a; mkmgsgp=RQXPUS'})
        html.encoding = 'cp1251'
        soup = BeautifulSoup(html.text, 'lxml')
        descr = soup.find('meta', attrs={"name": "Description", 'content' : True})
        if descr is not None:
            url_data[3] = descr['content']
        descr = soup.find('meta', attrs={"name": "description", 'content' : True})
        if descr is not None:
            url_data[3] = descr['content']
        del(descr)
        title = soup.find('title')
        if title is not None:
            url_data[2] = title.text
        del(title)
        keywords = soup.find('meta', attrs={"name": "keywords", 'content' : True})
        if keywords is not None:
            url_data[4] = keywords['content']
        keywords = soup.find('meta', attrs={"name": "Keywords", 'content' : True})
        if keywords is not None:
            url_data[4] = keywords['content']
        del(keywords)
        csv_writer(url_data)
        del(html)
        del(soup)     
        del(url_data)
    except AttributeError: 
        csv_writer(url_data)       
        del(url_data)
    except requests.exceptions.RequestException: 
        csv_writer(url_data)       
        del(url_data)
    except LocationParseError: 
        csv_writer(url_data)       
        del(url_data)      
i = 0
j = 5000
while (j < 14510):
    with Pool(40) as p: 
        p.map(parse_url, enumerate(no_lang[no_lang.index[range(i, j)]]))
    i = j
    print(i)
    if (j != 10000):
        j += 5000
    else:
        j = 14509

/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning
/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="x

5000


/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning
/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="x

10000


/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning
/opt/conda/lib/python3.7/site-packages/bs4/builder/__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="x

14509
CPU times: user 1.55 s, sys: 1.58 s, total: 3.13 s
Wall time: 11min 55s
